# Libraries

In [24]:
import os
import glob
import datetime

import numpy as np
import pandas as pd
from easyeda import eda

# Load Data

In [2]:
filedir = glob.glob(pathname='../Data/*.csv')[:-1]
filedir

['../Data\\area_passenger_index.csv',
 '../Data\\area_passenger_info.csv',
 '../Data\\grid_strength.csv',
 '../Data\\migration_index.csv',
 '../Data\\shortstay_20200117_20200131.csv',
 '../Data\\shortstay_20200201_20200215.csv']

In [3]:
area_passenger_ind = pd.read_csv(filedir[0],header = None)
area_passenger_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717840 entries, 0 to 717839
Data columns (total 3 columns):
0    717840 non-null int64
1    717840 non-null int64
2    717840 non-null float64
dtypes: float64(1), int64(2)
memory usage: 16.4 MB


In [5]:
area_passenger_ind.columns = ['gridIdx','datetime','Density']
area_passenger_ind['datetime'] = area_passenger_ind['datetime'].map(lambda x:str(x) + '00')
area_passenger_ind['datetime'] = pd.to_datetime(area_passenger_ind['datetime'])

# First attempt

In [6]:
area_passenger_ind['month'] = area_passenger_ind['datetime'].map(lambda x:x.month)
area_passenger_ind['day'] = area_passenger_ind['datetime'].map(lambda x:x.day)

cond1 = area_passenger_ind['month'] == 2
cond2 = area_passenger_ind['day'] >= 7

historic_pred = area_passenger_ind[cond1 & cond2]['Density']

In [7]:
submit_example = pd.read_csv('../Data/test_submit_example.csv',header = None)
submit_example.iloc[:,2] = list(historic_pred)
submit_example.to_csv('../Data/submit_historic_07_15.csv',index=None,header=None)

In [ ]:
submit_example.columns = ['gridIdx','datetime','Density']
submit_example['datetime'] = submit_example['datetime'].map(lambda x:str(x) + '00')
submit_example['datetime'] = pd.to_datetime(submit_example['datetime'])

submit_example['month'] = submit_example['datetime'].map(lambda x:x.month)
submit_example['day'] = submit_example['datetime'].map(lambda x:x.day)

In [29]:
mean = submit_example.pivot_table(index='gridIdx',
                           values='Density',
                           aggfunc='mean').reset_index()
mean.columns = ['gridIdx','MeanDensity']
mean['MeanDensity'] = 0.2 * np.array(mean['MeanDensity']) * np.random.rand(len(mean)) 
submit_example['Density'] = submit_example['Density'] + mean['MeanDensity']

In [30]:
noisy_historic_pred = submit_example['Density']

submit_example = pd.read_csv('../Data/test_submit_example.csv',header = None)
submit_example.iloc[:,2] = list(noisy_historic_pred)
submit_example.to_csv('../Data/submit_noisy_historic_07_15.csv',index=None,header=None)